In [1]:
%load_ext autoreload
%autoreload 2

In [12]:
from final_project.Trading_Project.utils.errors_handler import error_tracking_decorator, get_root_cause_error
from final_project.Trading_Project.utils.errors_handler.clean_error_message import clean_string, process_string, get_my_code_error

def plus(a, b):
    return a + b


@error_tracking_decorator
def my_func():
    result = plus("1",2)

r, s = my_func()
s

'Error in function my_func \n===================================================\nFilename: /tmp/ipykernel_178619/3668953505.py,\nFunction name: my_func, params: {}\n\t-----\n\tLine: 10,     result = plus("1",2)\n \t-----\n\tNote: This error is from your code\n===================================================\nFilename: /tmp/ipykernel_178619/3668953505.py,\nFunction name: plus, params: {\'a\': \'1\', \'b\': 2}\n\t-----\n\tLine: 5,     return a + b\n \t-->ROOT CAUSE: TypeError: can only concatenate str (not "int") to str \n\t-----\n\tNote: This error is from your code\n'

In [8]:
ss = [('\n'.join([ssi for ssi in s.split('\n') if '-----' not in ssi and 'Filename' not in ssi])).strip().replace('\t', '') for s in s.split("===================================================")]
ss


['Error in function my_func',
 'Function name: my_func, params: {}\nLine: 10,     result = plus("1",2)\nNote: This error is from your code',
 'Function name: plus, params: {\'a\': \'1\', \'b\': 2}\nLine: 5,     return a + b\n -->ROOT CAUSE: TypeError: can only concatenate str (not "int") to str \nNote: This error is from your code']

In [4]:
k = {
    "a": 1
}

if "a" in k:
    print(1)

1


In [4]:
import re

def clean_error_message(input_str):

    
    # Replace the matched pattern with an empty string
    cleaned_str = re.sub(r'\nNote: .*', '', input_str)
    
    return cleaned_str

# Test cases
input1 = 'Function name: my_func, params: {}\nLine: 10,     result = plus("1",2)\nNote: This error is from site package'
output1 = clean_error_message(input1)

input2 = 'Function name: plus, params: {\'a\': \'1\', \'b\': 2}\nLine: 5,     return a + b\n -->ROOT CAUSE: TypeError: can only concatenate str (not "int") to str \nNote: This error is from your code'
output2 = clean_error_message(input2)

output1, output2


('Function name: my_func, params: {}\nLine: 10,     result = plus("1",2)',
 'Function name: plus, params: {\'a\': \'1\', \'b\': 2}\nLine: 5,     return a + b\n -->ROOT CAUSE: TypeError: can only concatenate str (not "int") to str ')

In [13]:

processed_strings = [process_string(clean_string(string)) for string in s.split("===================================================")]
processed_strings = get_my_code_error(processed_strings)
processed_strings


['Function name: my_func, params: {}\nLine: 10,     result = plus("1",2)']

In [9]:
from final_project.Trading_Project.utils.file_text_handler import get_code_from_text
import backtrader as bt
from final_project.Trading_Project.base_strategy import BaseStrategy


# exec(get_code_from_text("```python\nimport backtrader as bt\n\nclass CustomIchimokuIndicator(bt.Indicator):\n    \"\"\"\n    Custom Ichimoku Cloud indicator.\n    \"\"\"\n    lines = (\"tenkan\", \"kijun\", \"senkou_a\", \"senkou_b\")\n    params = ((\"tenkan_period\", 9), (\"kijun_period\", 26), (\"senkou_period\", 52))\n\n    def __init__(self):\n        self.addminperiod(self.params.tenkan_period)\n        self.addminperiod(self.params.kijun_period)\n        self.addminperiod(self.params.senkou_period)\n\n    def next(self):\n        high = self.data.high\n        low = self.data.low\n        close = self.data.close\n\n        tenkan = (high.rolling(window=self.params.tenkan_period).max() + low.rolling(window=self.params.tenkan_period).min()) / 2\n        kijun = (high.rolling(window=self.params.kijun_period).max() + low.rolling(window=self.params.kijun_period).min()) / 2\n        senkou_a = (tenkan + kijun) / 2\n        senkou_b = (high.rolling(window=self.params.senkou_period).max() + low.rolling(window=self.params.senkou_period).min()) / 2\n\n        self.lines.tenkan[0] = tenkan[0]\n        self.lines.kijun[0] = kijun[0]\n        self.lines.senkou_a[0] = senkou_a[0]\n        self.lines.senkou_b[0] = senkou_b[0]\n\n\nclass BackTestStrategy(BaseStrategy):\n    def __init__(self, *args, **kwargs):\n        super().__init__(*args, **kwargs)\n        self.ichimoku = CustomIchimokuIndicator()\n\n    def execute(self):\n        \"\"\"\n        Define the trading logic based on the Ichimoku Cloud indicator.\n\n        Returns:\n        int: Trading signal: 1 (long), -1 (sell), or None if no signal.\n        \"\"\"\n        if self.ichimoku.tenkan[0] > self.ichimoku.kijun[0] and self.ichimoku.senkou_a[0] > self.ichimoku.senkou_b[0]:\n            return 1  # Long signal\n        elif self.ichimoku.tenkan[0] < self.ichimoku.kijun[0] and self.ichimoku.senkou_a[0] < self.ichimoku.senkou_b[0]:\n            return -1  # Short signal\n        return None  # No signal\n```"), globals())


import math

class GannAngle(bt.Indicator):
    lines = ("angle",)
    params = (("period", 1),)

    def __init__(self):
        self.addminperiod(self.params.period)

    def next(self):
        if len(self) == 1:
            self.lines.angle[0] = 0  # Initialize Gann Angle at the beginning
        else:
            high = self.data.high[0]
            low = self.data.low[0]
            close = self.data.close[0]
            self.lines.angle[0] = math.atan((high - low) / close) * (180 / math.pi)

class BackTestStrategy(BaseStrategy):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        # Initialize the Gann Angle indicator
        self.gann_angle = GannAngle()
        # Initialize the RSI indicator
        self.rsi = bt.indicators.RSI_SMA(self.data.close, period=14)

    def execute(self):
        """
        Define the trading logic based on the Gann Angle and RSI indicators.

        Returns:
        int: Trading signal: 1 (long), -1 (sell), or None if no signal.
        """
        if self.gann_angle[0] > 45 and self.rsi[0] > 50:  # Buy condition: Gann Angle indicates support and RSI is above 50
            return 1  # Long signal
        elif self.gann_angle[0] < 45 and self.rsi[0] < 50:  # Sell condition: Gann Angle indicates resistance and RSI is below 50
            return -1  # Short signal
        return None  # No signal

In [6]:
from final_project.Trading_Project.data_loader import load_stock_data

data = [bt.feeds.PandasData(
                dataname=load_stock_data(ticker='AAPL', period="1y"), datetime="Date", 
                timeframe=bt.TimeFrame.Minutes)]



[*********************100%%**********************]  1 of 1 completed


In [10]:
from final_project.Trading_Project.my_strategy import BackTestStrategy

class MyBuySell(bt.observers.BuySell):
    plotlines = dict(
        buy=dict(marker='$\u21E7$', markersize=12.0),
        sell=dict(marker='$\u21E9$', markersize=12.0)
    )

class CelebroCreator:
    def __init__(self, strategy, list_of_data, stake=100, cash=20000):
        self.cerebro = bt.Cerebro(cheat_on_open=True)

        self.cerebro.addstrategy(strategy)
        self.cerebro.addanalyzer(bt.analyzers.PyFolio, _name='pyfolio')
        self.cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='SharpeRatio')
        self.cerebro.addanalyzer(bt.analyzers.Returns, _name='Returns')
        self.cerebro.addobserver(bt.observers.Value)

        for data in list_of_data:
            self.cerebro.adddata(data)
        self.cerebro.broker.set_cash(cash)
        bt.observers.BuySell = MyBuySell
        
        # PREVIOUS
        # self.strats = self.cerebro.run()
        
        # CURRENT
        self.strats, self.message = self._run_cerebro()

    
    # CURRENT
    @error_tracking_decorator
    def _run_cerebro(self):
        self.cerebro.run()


    def show(self):
        print("Final Portfolio Value: %.2f" % self.cerebro.broker.getvalue())
        print("Total point return: ", (self.cerebro.broker.getvalue() - self.cerebro.broker.startingcash))

        try:
            sharpe_ratio = self.strats[0].analyzers.SharpeRatio.get_analysis()['sharperatio']
        except:
            print('No Buy/Sell Signal! No Sharpe Ratio!')
        # Plot the results
        figs = self.cerebro.plot(
            iplot=False, 
            # style="pincandle", 
            # width=60 * 10, height=40 * 10,
            figsize=(100, 80),
            # sharpe_ratio=sharpe_ratio
        )

# def check_valid_code(strategy, list_data):
    
#     @error_tracking_decorator
#     def extract_error_message(strategy, list_data):
#         CelebroCreator(strategy,list_data)
        
#     message = extract_error_message(BackTestStrategy, data)
#     root_error = get_root_cause_error(message) if message else ""
#     return root_error
def check_valid_code(strategy, list_data):
    
    obj = CelebroCreator(strategy,list_data)
        
    message = obj.message
    print(message)
    root_error = get_root_cause_error(message) if message else ""
    return root_error

error = check_valid_code(BackTestStrategy, data)

Error in function _run_cerebro 
Filename: /tmp/ipykernel_79758/2660629132.py,
Function name: _run_cerebro, params: {}
	-----
	Line: 34,         self.cerebro.run()
 	-----
	Note: This error is from your code
Filename: /home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/backtrader/cerebro.py,
Function name: run, params: {'kwargs': {}, 'pkeys': "odict_keys(['preload', 'runonce'...", 'iterstrats': '<itertools.product object at 0x7...', 'iterstrat': "((<class 'final_project.Trading_..."}
	-----
	Line: 1132,                 runstrat = self.runstrategies(iterstrat)
 	-----
	Note: This error is from site-packages
Filename: /home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/backtrader/cerebro.py,
Function name: runstrategies, params: {'iterstrat': "((<class 'final_project.Trading_...", 'predata': False, 'runstrats': '[<final_project.Trading_Project....', 'data': '<backtrader.feeds.pandafeed.Pand...', 'stratcls': "<class 'final_project.Trading_Pr...", 'sargs': None

In [3]:
print("import backtrader as bt\nimport numpy as np\n\nclass GannAngle(bt.Indicator):\n    lines = (\"angle\",)\n    params = ((\"period\", 1),)\n\n    def __init__(self):\n        self.addminperiod(self.params.period)\n\n    def next(self):\n        # Calculate Gann Angle (example implementation, may need adjustment)\n        self.lines.angle[0] = self.data.close[0] / self.data.close[-self.params.period]\n\nclass RSI(bt.Indicator):\n    lines = (\"rsi\",)\n    params = ((\"period\", 14),)\n\n    def __init__(self):\n        self.addminperiod(self.params.period)\n        self.up, self.down = 0, 0\n\n    def next(self):\n        # Calculate RSI (example implementation, may need adjustment)\n        delta = self.data.close[0] - self.data.close[-1]\n        if delta > 0:\n            self.up += delta\n        else:\n            self.down -= delta\n\n        if len(self) >= self.params.period:\n            self.lines.rsi[0] = 100 - 100 / (1 + self.up / self.down)\n            self.up, self.down = 0, 0\n        else:\n            self.lines.rsi[0] = 0\n\nclass BackTestStrategy(BaseStrategy):\n    def __init__(self, *args, **kwargs):\n        super().__init__(*args, **kwargs)\n        # Initialize Gann Angle and RSI indicators\n        self.gann_angle = GannAngle()\n        self.rsi = RSI()\n\n    def execute(self):\n        \"\"\"\n        Define the trading logic based on Gann Angles and RSI indicators.\n\n        Returns:\n        int: Trading signal: 1 (long), -1 (sell), or None if no signal.\n        \"\"\"\n        if self.gann_angle[0] > 0 and self.rsi[0] > 50:\n            # Buy condition: Gann Angle indicates support and RSI is above 50\n            return 1  # Long signal\n        elif self.gann_angle[0] < 0 and self.rsi[0] < 50:\n            # Sell condition: Gann Angle indicates resistance and RSI is below 50\n            return -1  # Short signal\n        return None  # No signal\n")

import backtrader as bt
import numpy as np

class GannAngle(bt.Indicator):
    lines = ("angle",)
    params = (("period", 1),)

    def __init__(self):
        self.addminperiod(self.params.period)

    def next(self):
        # Calculate Gann Angle (example implementation, may need adjustment)
        self.lines.angle[0] = self.data.close[0] / self.data.close[-self.params.period]

class RSI(bt.Indicator):
    lines = ("rsi",)
    params = (("period", 14),)

    def __init__(self):
        self.addminperiod(self.params.period)
        self.up, self.down = 0, 0

    def next(self):
        # Calculate RSI (example implementation, may need adjustment)
        delta = self.data.close[0] - self.data.close[-1]
        if delta > 0:
            self.up += delta
        else:
            self.down -= delta

        if len(self) >= self.params.period:
            self.lines.rsi[0] = 100 - 100 / (1 + self.up / self.down)
            self.up, self.down = 0, 0
        else:
            self.line

In [51]:
class Foo:
    # result = 0
    def __init__(self, a=3, b=2):
        self.a = a
        self.b = b
        # Foo.result = 0
        self.sum()


    def sum(self):
        Foo.result = self.a + self.b
    
    @classmethod
    def get_result(cls):
        return cls.result

    


k = Bar

result = k.get_result()
result

AttributeError: type object 'Bar' has no attribute 'result'

In [53]:
print("python\nimport backtrader as bt\nimport numpy as np\n\nclass GannAngle(bt.Indicator):\n    lines = (\"angle\",)\n    params = ((\"period\", 1),)\n\n    def __init__(self):\n        self.addminperiod(self.params.period)\n\n    def next(self):\n        # Calculate Gann Angle (example implementation, may need adjustment)\n        self.lines.angle[0] = self.data.close[0] / self.data.close[-1] * self.params.period\n\nclass RSI(bt.Indicator):\n    lines = (\"rsi\",)\n    params = ((\"period\", 14),)\n\n    def __init__(self):\n        self.addminperiod(self.params.period)\n\n    def next(self):\n        # Calculate RSI (example implementation, may need adjustment)\n        delta = np.array(self.data.close[0] - self.data.close[-1])\n        up, down = delta.copy(), delta.copy()\n        up[up < 0] = 0\n        down[down > 0] = 0\n        roll_up = bt.indicators.SumN(up, period=self.params.period)\n        roll_down = bt.indicators.SumN(down, period=self.params.period).abs()\n        self.lines.rsi[0] = 100.0 - 100.0 / (1.0 + roll_up / roll_down)\n\nclass BackTestStrategy(BaseStrategy):\n    def __init__(self, *args, **kwargs):\n        super().__init__(*args, **kwargs)\n        self.gann_angle = GannAngle()\n        self.rsi = RSI()\n\n    def execute(self):\n        if self.gann_angle[0] > 0 and self.rsi[0] > 50:\n            # Buy condition: Gann Angle indicates support and RSI is above 50\n            return 1  # Long signal\n        elif self.gann_angle[0] < 0 and self.rsi[0] < 50:\n            # Sell condition: Gann Angle indicates resistance and RSI is below 50\n            return -1  # Short signal\n        return None  # No signal\n")

python
import backtrader as bt
import numpy as np

class GannAngle(bt.Indicator):
    lines = ("angle",)
    params = (("period", 1),)

    def __init__(self):
        self.addminperiod(self.params.period)

    def next(self):
        # Calculate Gann Angle (example implementation, may need adjustment)
        self.lines.angle[0] = self.data.close[0] / self.data.close[-1] * self.params.period

class RSI(bt.Indicator):
    lines = ("rsi",)
    params = (("period", 14),)

    def __init__(self):
        self.addminperiod(self.params.period)

    def next(self):
        # Calculate RSI (example implementation, may need adjustment)
        delta = np.array(self.data.close[0] - self.data.close[-1])
        up, down = delta.copy(), delta.copy()
        up[up < 0] = 0
        down[down > 0] = 0
        roll_up = bt.indicators.SumN(up, period=self.params.period)
        roll_down = bt.indicators.SumN(down, period=self.params.period).abs()
        self.lines.rsi[0] = 100.0 - 100.0 / (1.0 + roll

In [2]:
%pip install -qU langchain-openai

Note: you may need to restart the kernel to use updated packages.


In [12]:
import os
from langchain_openai import ChatOpenAI
os.environ['ANYSCALE_API_KEY'] = "esecret_bn1yi2ygtils5v241f56sah5el"
os.environ['ANYSCALE_API_BASE'] = "https://api.endpoints.anyscale.com/v1"

llm = ChatOpenAI(
    model="meta-llama/Meta-Llama-3-70B-Instruct",
    temperature=0,
    max_tokens=2048,
    # max_retries=2,
    api_key=os.environ['ANYSCALE_API_KEY'],
    base_url=os.environ['ANYSCALE_API_BASE'],
)


query = f""" With list technical indicators show below
**Volatility Indicators**

1. Keltner Channel
2. Donchian Channel
3. Historical Volatility (HV)

**Oscillators**

1. Commodity Channel Index (CCI)
2. Williams %R
3. Lane's Stochastic Oscillator
4. Detrended Price Oscillator (DPO)

**Cycle Indicators**

1. Schaff Trend Cycle (STC)
2. Cycle Identifier
3. Hurst Exponential Smoothing (HES)

**Other Indicators**

1. Fibonacci Retracement
2. Fibonacci Extension
3. Pivot Points
4. Camarilla Pivot Points
5. Demark Indicators (TDI, TD Sequential)
6. Elder's Three Screens System
7. Kagi Chart
8. Renko Chart
9. Point and Figure Chart

Show the formula for each indicators. If which indicator you don't know, please just show message "I DON'T KNOW".
"""

messages = [
    (
        "system",
        "You are a helpful assistant.",
    ),
    ("human", query),
]
ai_msg = llm.invoke(messages)
ai_msg

AIMessage(content="Here are the formulas for each technical indicator:\n\n**Volatility Indicators**\n\n1. Keltner Channel:\n\t* Upper Channel: EMA(high, n) + (2 * ATR(n))\n\t* Lower Channel: EMA(low, n) - (2 * ATR(n))\n\t* Middle Channel: EMA(close, n)\n\t* Where n is the period, EMA is the Exponential Moving Average, and ATR is the Average True Range\n2. Donchian Channel:\n\t* Upper Channel: Highest high for n periods\n\t* Lower Channel: Lowest low for n periods\n\t* Middle Channel: (Upper Channel + Lower Channel) / 2\n3. Historical Volatility (HV):\n\t* HV = Stdev(ln(close / close(-1))) * sqrt(252)\n\t* Where Stdev is the standard deviation, ln is the natural logarithm, and close(-1) is the previous period's close price\n\n**Oscillators**\n\n1. Commodity Channel Index (CCI):\n\t* CCI = (Typical Price - MA(Typical Price, n)) / (0.015 * Mean Deviation)\n\t* Where Typical Price = (High + Low + Close) / 3, MA is the Moving Average, and Mean Deviation is the average absolute deviation fro

In [13]:
print(ai_msg)

content="Here are the formulas for each technical indicator:\n\n**Volatility Indicators**\n\n1. Keltner Channel:\n\t* Upper Channel: EMA(high, n) + (2 * ATR(n))\n\t* Lower Channel: EMA(low, n) - (2 * ATR(n))\n\t* Middle Channel: EMA(close, n)\n\t* Where n is the period, EMA is the Exponential Moving Average, and ATR is the Average True Range\n2. Donchian Channel:\n\t* Upper Channel: Highest high for n periods\n\t* Lower Channel: Lowest low for n periods\n\t* Middle Channel: (Upper Channel + Lower Channel) / 2\n3. Historical Volatility (HV):\n\t* HV = Stdev(ln(close / close(-1))) * sqrt(252)\n\t* Where Stdev is the standard deviation, ln is the natural logarithm, and close(-1) is the previous period's close price\n\n**Oscillators**\n\n1. Commodity Channel Index (CCI):\n\t* CCI = (Typical Price - MA(Typical Price, n)) / (0.015 * Mean Deviation)\n\t* Where Typical Price = (High + Low + Close) / 3, MA is the Moving Average, and Mean Deviation is the average absolute deviation from the mean

In [14]:
print("**Volatility Indicators**\n\n1. Keltner Channel:\n\t* Upper Channel: EMA(high, n) + (2 * ATR(n))\n\t* Lower Channel: EMA(low, n) - (2 * ATR(n))\n\t* Middle Channel: EMA(close, n)\n\t* Where n is the period, EMA is the Exponential Moving Average, and ATR is the Average True Range\n2. Donchian Channel:\n\t* Upper Channel: Highest high for n periods\n\t* Lower Channel: Lowest low for n periods\n\t* Middle Channel: (Upper Channel + Lower Channel) / 2\n3. Historical Volatility (HV):\n\t* HV = Stdev(ln(close / close(-1))) * sqrt(252)\n\t* Where Stdev is the standard deviation, ln is the natural logarithm, and close(-1) is the previous period's close price\n\n**Oscillators**\n\n1. Commodity Channel Index (CCI):\n\t* CCI = (Typical Price - MA(Typical Price, n)) / (0.015 * Mean Deviation)\n\t* Where Typical Price = (High + Low + Close) / 3, MA is the Moving Average, and Mean Deviation is the average absolute deviation from the mean\n2. Williams %R:\n\t* %R = (Highest high for n periods - Close) / (Highest high for n periods - Lowest low for n periods)\n3. Lane's Stochastic Oscillator:\n\t* %K = (Close - Lowest low for n periods) / (Highest high for n periods - Lowest low for n periods)\n\t* %D = SMA(%K, n)\n\t* Where SMA is the Simple Moving Average\n4. Detrended Price Oscillator (DPO):\n\t* DPO = Close - SMA(Close, n)\n\t* Where SMA is the Simple Moving Average\n\n**Cycle Indicators**\n\n1. Schaff Trend Cycle (STC):\n\t* I DON'T KNOW (The STC is a proprietary indicator, and its formula is not publicly available)\n2. Cycle Identifier:\n\t* I DON'T KNOW (I couldn't find a widely recognized formula for this indicator)\n3. Hurst Exponential Smoothing (HES):\n\t* I DON'T KNOW (HES is a complex indicator that involves multiple calculations, and I couldn't find a concise formula to provide)\n\n**Other Indicators**\n\n1. Fibonacci Retracement:\n\t* No formula, as it's a graphical tool used to identify potential support and resistance levels\n2. Fibonacci Extension:\n\t* No formula, as it's a graphical tool used to identify potential price targets\n3. Pivot Points:\n\t* PP = (High + Low + Close) / 3\n\t* R1 = PP + (High - Low)\n\t* R2 = PP + (High - Low) * 2\n\t* R3 = PP + (High - Low) * 3\n\t* S1 = PP - (High - Low)\n\t* S2 = PP - (High - Low) * 2\n\t* S3 = PP - (High - Low) * 3\n4. Camarilla Pivot Points:\n\t* R3 = Close + ((High - Low) * 1.1)\n\t* R4 = Close + ((High - Low) * 1.4)\n\t* R5 = Close + ((High - Low) * 1.6)\n\t* S3 = Close - ((High - Low) * 1.1)\n\t* S4 = Close - ((High - Low) * 1.4)\n\t* S5 = Close - ((High - Low) * 1.6)\n5. Demark Indicators (TDI, TD Sequential):\n\t* I DON'T KNOW (These indicators are proprietary, and their formulas are not publicly available)\n6. Elder's Three Screens System:\n\t* I DON'T KNOW (This is a trading system that involves multiple indicators and rules, and I couldn't find a concise formula to provide)\n7. Kagi Chart:\n\t* No formula, as it's a graphical charting method that uses vertical lines to represent price movements\n8. Renko Chart:\n\t* No formula, as it's a graphical charting method that uses bricks to represent price movements\n9. Point and Figure Chart:\n\t* No formula, as it's a graphical charting method that uses X's and O's to represent price movements")

**Volatility Indicators**

1. Keltner Channel:
	* Upper Channel: EMA(high, n) + (2 * ATR(n))
	* Lower Channel: EMA(low, n) - (2 * ATR(n))
	* Middle Channel: EMA(close, n)
	* Where n is the period, EMA is the Exponential Moving Average, and ATR is the Average True Range
2. Donchian Channel:
	* Upper Channel: Highest high for n periods
	* Lower Channel: Lowest low for n periods
	* Middle Channel: (Upper Channel + Lower Channel) / 2
3. Historical Volatility (HV):
	* HV = Stdev(ln(close / close(-1))) * sqrt(252)
	* Where Stdev is the standard deviation, ln is the natural logarithm, and close(-1) is the previous period's close price

**Oscillators**

1. Commodity Channel Index (CCI):
	* CCI = (Typical Price - MA(Typical Price, n)) / (0.015 * Mean Deviation)
	* Where Typical Price = (High + Low + Close) / 3, MA is the Moving Average, and Mean Deviation is the average absolute deviation from the mean
2. Williams %R:
	* %R = (Highest high for n periods - Close) / (Highest high for n periods - 